# Profiling
### Finding bottlenecks in your Python program
Simon Funke

Date: **Sep 28, 2016**

## Contents

* Timing
* Profiling
* Line-by-line profiling

  *"First make it work. Then make it right. Then make it fast."* 
  
  Kent Beck

Prior to code optimization, hotspots and bottlenecks must be located. 

## Profiling allows us to measure resources used by sections of the program. 

Typical resources of interest are
* Amount of CPU used
* Amount of RAM used

But also other resources can be measured, such as:

* Disk I/O
* Network I/O
* ...

## Profiling techniques

Start simple, switch to more complex techniques if needed!

Techniques for measuring CPU time:
1. Manual timeing
2. `timeit` module
3. `profile` or `cprofile` modules

## Case study: filling a grid with point values

* Consider a rectangular 2D grid
<center>![xkcd](pdf/grid_lattice.svg "Python")Grid lattice</center>
* A NumPy array `a[i,j]` holds values at the grid points

# An implementation

In [2]:
from numpy import *

class Grid2D(object):
    def __init__(self,
                 xmin=0, xmax=1, dx=0.5,
                 ymin=0, ymax=1, dy=0.5):
        
        self.xcoor = arange(xmin, xmax+dx, step=dx)
        self.ycoor = arange(ymin, ymax+dy, step=dy)

    def gridloop(self, f):
        lx = size(self.xcoor)
        ly = size(self.ycoor)
        a = zeros((lx,ly))

        for i in range(lx):
            x = self.xcoor[i]
            for j in range(ly):
                y = self.ycoor[j]
                a[i,j] = f(x, y)
        return a

# Usage

Create a new grid:

In [3]:
g = Grid2D(dx=0.001, dy=0.001)

Computing grid values:

In [4]:
import numpy
def myfunc(x, y):
    return numpy.sin(x*y) + y

print("Computing values...")
a = g.gridloop(myfunc)
print("done")

Computing values...
done


Getting grid values at a specific point:

In [5]:
i=1000
j=1000
print('value at ({}, {}) is {}'.format(g.xcoor[i], g.ycoor[j], a[i,j]))

value at (1.0, 1.0) is 1.8414709848078965


# Timing

Use `time.time()` to measure the time spend in a code section.
  ```python
  t0 = time.time()
  # execute code here
  t1 = time.time()
  print("Runtime: {}".format(t1-t0))
  ```
  

* Simple statements should be placed in a loop.
* Make sure to use a constant machine load.
* Run the tests several times, choose the fastest.

*Note*: `time.time` measures wall clock time. Use `time.clock()` to measure CPU time.

## Timing of the case study

The case study has two parts that could potentially be slow: 
1. The initialisation `Grid2D(dx=0.001, dy=0.001)`
2. Calling the `g.gridloop(myfunc)` function.

We time these two parts separately to figure out how much time is spend in each.

### Timing the Grid2D initialisation

In [6]:
import time

for i in range(1, 4):
    t0 = time.time()
    g = Grid2D(dx=0.001, dy=0.001)
    t1 = time.time()
    print("Experiment {}, CPU time: {:.4} s".format(i, t1-t0))
print("Done")

Experiment 1, CPU time: 0.002901 s
Experiment 2, CPU time: 0.0001416 s
Experiment 3, CPU time: 2.313e-05 s
Done


### Timing the `gridloop` function

In [7]:
import time

for i in range(1, 4):
    t0 = time.time()
    g.gridloop(myfunc)
    t1 = time.time()
    print("Experiment {}. CPU time: {:.4} s".format(i, t1-t0))
print("Done")

Experiment 1. CPU time: 1.52 s
Experiment 2. CPU time: 1.958 s
Experiment 3. CPU time: 1.91 s
Done


$=>$ The gridloop function is the cause of the slow execution!

# The *timeit* module

## The *timeit* module (1)

The `timeit` module provides an convienient way for measuring the CPU time of small code snippets.

Usage:

In [10]:
import timeit
timeit.timeit(stmt="a+=1",setup="a=0")

0.07175554199989165

Adding the `number` keyword executes the statement multiple times and returns the **accumulated** runtime:

In [11]:
timeit.timeit(stmt="a+=1",setup="a=0", number=10000)

0.0005505629999333905

Use `timeit.repeat` if you would like to repeat the experiment multiple times:

In [12]:
timeit.repeat(stmt="a+=1",setup="a=0", number=10000, repeat=5)

[0.0007694490000176302,
 0.0005602549999821349,
 0.0005591609999555658,
 0.0004939969999213645,
 0.000502825000012308]

## The *timeit* module (2)

* Automatically wraps the code in a for - loop.

* Users can provide their own timer (callback):

In [13]:
timeit.timeit(stmt="sleep(1)", setup="from time import sleep", number=1, timer=time.time)  # Wall-clock time

1.0009279251098633

In [14]:
timeit.timeit(stmt="sleep(1)", setup="from time import sleep", number=1, timer=time.clock)  # CPU time

0.0013140000000007035

* Isolates the global namespace.

* Time a user defined function:

In [15]:
timeit.repeat(stmt="g.gridloop(myfunc)", setup="from __main__  import g, myfunc", repeat=5, number=1)

[1.5012751060000937,
 1.5762523280000096,
 1.4885704539999551,
 1.4240629760001866,
 1.3900808190001044]

## Profiling modules

A profile is a set of statistics that describes how often and for how long various parts of the program executed.

There are two main (deterministic) profiling modules in Python

* `profile`, written purely in Python
* `cProfile`, a C implementation introduced in Python 2.5.

The statistics from both can be formatted into reports via the `pstats` module.    

## Using the `profile` module

* As a script: `python -m profile script.py
`

* As a module:

In [13]:
import profile
pr = profile.Profile()
res = pr.run("g.gridloop(myfunc)")
pr.dump_stats("gridloop.prof")  # Dump statistics to file for use with pstats

## Calibrate to avoid measuring CPU overhead

The `profile` module needs to compensate for CPU overhead caused by internal Python function calls:

   1. Find the profiling overhead: 
     ```python
     overhead = pr.calibrate(number)
     ```
   2. Remove profiling overhead:
      ```python
      pr = profile.Profile(bias=overhead)
      ```

## Profile a function call while keeping its return value

```python
pr = profile.Profile()
retval = pr.runcall(func, *args, **kwargs)
```

## Getting runtime statistics

* We can view some runtime statistics with

In [14]:
res.print_stats()

         1002009 function calls in 7.304 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    7.304    7.304 :0(exec)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        1    0.002    0.002    0.002    0.002 :0(zeros)
        1    2.977    2.977    7.304    7.304 <ipython-input-1-af8978036994>:11(gridloop)
  1002001    4.324    0.000    4.324    0.000 <ipython-input-3-a92c08ef78a2>:2(myfunc)
        1    0.000    0.000    7.304    7.304 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:2659(size)
        1    0.000    0.000    7.304    7.304 profile:0(g.gridloop(myfunc))
        0    0.000             0.000          profile:0(profiler)




## Statistics header definitions
* **ncalls**: number of calls
* **tottime**: total time spent in the given function excluding time made in calls to sub-functions
* **percall**: tottime divided by ncalls
* **cumtime**: cumulative time spent in this and all subfunctions
* **percall**: cumtime divided by ncalls
* **filename:lineno(function)**: information about the function

## The `cProfile` module

* Similar to `profile`, but mostly implemented in C.
* Smaller performance impact than `profile`. Overhead calibaration not required.

The usage is compatible with that of `profile`:
```python
import cProfile
cProfile.run('foo()', 'gridloop.prof')
```

or to profile a script:
```bash
python -m cProfile my_script.py
```

## The `pstats` module

There are many ways to view profiling data.

The module `pstats` provides the class `Stats` for creating profiling reports:

In [15]:
import pstats
stats = pstats.Stats("gridloop.prof")
stats.sort_stats("time")
stats.print_stats(3)

Wed Sep 28 07:57:58 2016    gridloop.prof

         1002009 function calls in 7.304 seconds

   Ordered by: internal time
   List reduced from 9 to 3 due to restriction <3>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1002001    4.324    0.000    4.324    0.000 <ipython-input-3-a92c08ef78a2>:2(myfunc)
        1    2.977    2.977    7.304    7.304 <ipython-input-1-af8978036994>:11(gridloop)
        1    0.002    0.002    0.002    0.002 :0(zeros)




* The method `sort_stats(key, *keys)` is used to sort future output.
* Common used keys: `'calls', 'cumulative', 'time'`.
* The argument to `print_stats` limits the view to the N most significant results.

## Back to our case-study: What have we learned so far?

The total runtime (with `profile`) is 8.1s. Biggest contributors:
   1. `gridloop` contributes ca 41% of the total runtime.
   2. `myfunc` calls contributes ca 55% of the total runtime.

* `myfunc` is fairly straight-forward
  ```python
  def myfunc(x, y):
    return sin(x*y) + y
  ```
  Might be difficult to improve.
* What about `gridloop`?

## Recall that, `gridloop` was a function of the form

```python
def gridloop(self, f):
    lx = size(self.xcoor)
    ly = size(self.ycoor)
    a = zeros((lx,ly))

    for i in range(lx):
        x = self.xcoor[i]
        for j in range(ly):
            y = self.ycoor[j]
             a[i,j] = f(x, y)
    return a
```

It would be useful to see how much time is spend in each line!

## Line by line profiling

The `line_profiler` inspects the time spend in each line of a Python function. 

## Usage

1. Install with `pip install line_profiler`
2. "Decorate" the function of interest with `@profile`:
    ```python
    @profile
    def gridloop(func):
        # ...
    ```
3. Run line profiler with:
    ```bash
    kernprof -l -v grid2d_lineprofile.py
    ```

## Demo

In [16]:
!kernprof -l -v grid2d_lineprofile.py

Wrote profile results to grid2d_lineprofile.py.lprof
Timer unit: 1e-06 s

Total time: 3.56572 s
File: grid2d_lineprofile.py
Function: gridloop at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                               @profile
    12                                               def gridloop(self, f):
    13         1           14     14.0      0.0          lx = size(self.xcoor)
    14         1            6      6.0      0.0          ly = size(self.ycoor)
    15         1           14     14.0      0.0          a = zeros((lx,ly))
    16                                           
    17      1002          645      0.6      0.0          for i in range(lx):
    18      1001         1075      1.1      0.0              x = self.xcoor[i]
    19   1003002       587556      0.6     16.5              for j in range(ly):
    20   1002001       708988      0.7     19.9                  y = self.ycoor[j]
    21   1002001      2267422      2.3  

**Conclusion:** Most of the time is spend in loops and indexing.

## A vectorised Grid2D implementation

In [18]:
class VectorisedGrid2D(object):
    def __init__(self,
                 xmin=0, xmax=1, dx=0.5,
                 ymin=0, ymax=1, dy=0.5):
        
        self.xcoor = arange(xmin, xmax+dx, step=dx)
        self.ycoor = arange(ymin, ymax+dy, step=dy)

    def gridloop(self, f):
        return f(self.xcoor[:,None], self.ycoor[None,:])  # Vectorized grid evaluation 

## Timing the vectorised Grid2D

In [19]:
vg = VectorisedGrid2D(dx=0.001, dy=0.001)
timeit.repeat(stmt="vg.gridloop(myfunc)", setup="from __main__  import vg, myfunc", repeat=5, number=1)

[0.04800678199990216,
 0.021695647000342433,
 0.01941766900017683,
 0.019448389999979554,
 0.01865350199977911]

In [20]:
g = Grid2D(dx=0.001, dy=0.001)
timeit.repeat(stmt="g.gridloop(myfunc)", setup="from __main__  import g, myfunc", repeat=5, number=1)

[1.478602740000042,
 1.4124204690001534,
 1.4405541979999725,
 1.4552482019998934,
 1.422352549000152]

**Vectorization yields a 70 times speed improvement!**

# Vectorization

In [26]:
a = numpy.ones(10)
b = numpy.arange(10)

In [27]:
print(a)

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [28]:
print(b)

[0 1 2 3 4 5 6 7 8 9]


In [29]:
a+b

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.])

## Non-vectorised implementation

In [34]:
out = ndarray(len(b))
for i in range(len(b)):
    out[i] = sin(b[i])

## Vectorised implementation

In [30]:
numpy.sin(b)

array([ 0.        ,  0.84147098,  0.90929743,  0.14112001, -0.7568025 ,
       -0.95892427, -0.2794155 ,  0.6569866 ,  0.98935825,  0.41211849])